In [1]:
import json
import pandas as pd
from transformers import BertTokenizerFast, BertForMultipleChoice
from datasets import load_dataset, concatenate_datasets
import datasets
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch
import evaluate
import numpy as np
from transformers import TrainingArguments, Trainer

from transformers import AlbertForMultipleChoice, AutoModelForMultipleChoice
import transformers
# transformers.logging.set_verbosity_error()
from transformers import logging
import csv
logging.set_verbosity_error()




c:\Users\Terry\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import opencc
cc = opencc.OpenCC('s2t.json')
cc.convert('汉字')  # 漢字

'漢字'

In [3]:
# dim1: n, index of sample
# dim2: 0, context ; 1, {question choice answer}*m ; 2, X
# dim3: 0, X       ; 1, {question choice answer}   ; 2, X
file = open('train_HW3dataset.json',encoding="utf-8")
train = json.load(file)
file.close()
file = open('dev_HW3dataset.json',encoding="utf-8")
dev = json.load(file)
file.close()
file = open('test_HW3dataset.json',encoding="utf-8")
test = json.load(file)
file.close()


In [4]:
test[0][1][0]['choice']

['小女孩怕麻烦', '小女孩认为孕妇怕麻烦', '孕妇怕麻烦', '孕妇很聪明']

test_index = [ [choice2_sample_index_lst] [choice3_sample_index_lst] [choice4_sample_index_lst] ]

In [5]:
test_index = [[], [], []]

def output_json(samples, outfile_name="train.json", test_file=False):
    data = []
    test_data = [[], [], []]

    index = 0
    for sample in samples:
        for qac in sample[1]:
            choice_lst = [ cc.convert(choice) for choice in qac['choice'] ]
            if not test_file:
                train_dict = {'context': cc.convert(sample[0][0]), 'question': cc.convert(qac['question']), 'label': qac['choice'].index(qac['answer']), 'choices': choice_lst}
                data.append(train_dict)
            else:
                train_dict = {'context': cc.convert(sample[0][0]), 'question': cc.convert(qac['question']), 'label': "",'choices': choice_lst}
                test_data[len(qac['choice']) - 2].append(train_dict)
                test_index[len(qac['choice']) - 2].append(index)
                index += 1

    if not test_file:
        output_dict = {"data": data}
        with open(outfile_name, "w") as outfile:
            json.dump(output_dict, outfile, indent=2)#, ensure_ascii=False)
    else:
        for num_choice in range(2,5):
            # data_choice = {k: v for k, v in data.items() if len(v['choices']) == num_choice }

            output_dict = {"data": test_data[num_choice-2]}
            with open(f"{outfile_name[:-5]}{num_choice}.json" , "w") as outfile:
                json.dump(output_dict, outfile, indent=2) #, ensure_ascii=False)


            


        
output_json(train, outfile_name="train.json")
output_json(dev, outfile_name="dev.json")
output_json(test, outfile_name="test.json", test_file=True)


In [6]:
file = open('test4.json',encoding="utf-8")
ttrain = json.load(file)
file.close()
ttrain['data'][100]


{'context': '在這個重視外表的時代，尤其是在文藝界裏，周杰倫的成功有點兒不可思議。就像他自己所說：“我大部分的女歌迷都不會對我說我很帥，相反地，她們會告訴我，她們喜歡我的音樂，被我的音樂吸引。”\n據母親回憶，周杰倫在學會走路前，就對音樂很敏感。周媽媽在他四歲的時候，就送他進鋼琴班學琴，而且他十分努力，對鋼琴很瘋狂。高中鋼琴老師說，周杰倫十幾歲時，就可以當衆表演。出了練琴的房問，周杰倫是個再普通不過的青少年，打籃球、看功夫電影、打遊戲，在學習上並不優秀，大學都很難考上。\n是音樂救了他，給他帶來了幸運。有一次，朋友幫他報名參加一個節目，他不敢一個人表演，決定幫一位想當歌手的朋友寫歌，併爲他彈鋼琴。那位想當歌手的朋友唱得很糟糕，但是主持人看了周杰倫寫的歌，立即找到他，與他簽了合同，讓他專門爲歌手寫歌。\n就這樣，他用兩年時間專心爲歌手寫歌，他寫的歌總能流行起來。後來，他又從幕後走到臺前，成爲亞洲最流行的歌手之一。',
 'question': '周杰倫對什麼很敏感?',
 'label': '',
 'choices': ['籃球', '功夫', '音樂', '遊戲']}

In [7]:
def split_by_choices(data_ds, num_choice):
    split_data_ds = data_ds.filter(lambda example:  len(example['choices']) == num_choice )
    return split_data_ds
    
data_ds = load_dataset("json", data_files={"train":"train.json", "dev": "dev.json"}, field="data")
data_ds2 = split_by_choices(data_ds, 2)
data_ds3 = split_by_choices(data_ds, 3)
data_ds4 = split_by_choices(data_ds, 4)

test_data_ds2 = load_dataset("json", data_files={"test":"test2.json"}, field="data")
test_data_ds3 = load_dataset("json", data_files={"test":"test3.json"}, field="data")
test_data_ds4 = load_dataset("json", data_files={"test":"test4.json"}, field="data")








Using custom data configuration default-fb3ad7adaa55915d


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 2004.93it/s]


Dataset json downloaded and prepared to C:/Users/Terry/.cache/huggingface/datasets/json/default-fb3ad7adaa55915d/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 118.04ba/s]
Using custom data configuration default-c153f55b5e3353c7


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1005.11it/s]


...

Dataset json downloaded and prepared to C:/Users/Terry/.cache/huggingface/datasets/json/default-b758766aec7fceb6/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 167.20it/s]
Using custom data configuration default-8b04f5dda9f24bb1


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1004.14it/s]


Dataset json downloaded and prepared to C:/Users/Terry/.cache/huggingface/datasets/json/default-8b04f5dda9f24bb1/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 167.21it/s]


In [8]:

test_data_ds4


DatasetDict({
    test: Dataset({
        features: ['context', 'question', 'label', 'choices'],
        num_rows: 1672
    })
})

In [9]:
# tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-bert-wwm")
tokenizer = BertTokenizerFast.from_pretrained("ckiplab/bert-base-chinese")


# print(tokenizer)


In [10]:

def preprocess_function(examples):
    num_choice = len(examples["choices"][0])
    print("num_choice = ", num_choice)

    contexts_questions = [[f"{context}[SEP]{question}"]*num_choice for context, question in zip(examples["context"], examples["question"])]
    choices = examples["choices"]
    contexts_questions = sum(contexts_questions, [])
    choices = sum(choices, [])

    tokenized_examples = tokenizer(contexts_questions, choices, truncation=True)
    
    # print(tokenized_examples.keys())
    
    return {k: [v[i : i + num_choice] for i in range(0, len(v), num_choice)] for k, v in tokenized_examples.items() }

# train
tokenized_data_ds2 = data_ds2.map(preprocess_function, batched=True)
tokenized_data_ds3 = data_ds3.map(preprocess_function, batched=True)
tokenized_data_ds4 = data_ds4.map(preprocess_function, batched=True)
# # test
# tokenized_test_data_ds2 = test_data_ds2.map(preprocess_function, batched=True)
# tokenized_test_data_ds3 = test_data_ds3.map(preprocess_function, batched=True)
# tokenized_test_data_ds4 = test_data_ds4.map(preprocess_function, batched=True)

100%|██████████| 1/1 [00:00<00:00, 13.38ba/s]


num_choice =  2


100%|██████████| 1/1 [00:00<00:00, 40.13ba/s]


num_choice =  2


  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

num_choice =  4


 50%|█████     | 1/2 [00:01<00:01,  1.02s/ba]

num_choice =  4


100%|██████████| 2/2 [00:01<00:00,  1.17ba/s]


In [11]:
tokenizer.decode(tokenized_data_ds4['train']['input_ids'][0][0])

'[CLS] 許 多 動 物 的 某 些 器 官 感 覺 特 別 靈 敏 ， 它 們 能 比 人 類 提 前 知 道 一 些 災 害 事 件 的 發 生 ， 例 如 ， 海 洋 中 的 水 母 能 預 報 風 暴 ， 老 鼠 能 事 先 躲 避 礦 井 崩 塌 或 有 害 氣 體 ， 等 等 。 地 震 往 往 能 使 一 些 動 物 的 某 些 感 覺 器 官 受 到 刺 激 而 發 生 異 常 反 應 。 如 一 個 地 區 的 重 力 發 生 變 異 ， 某 些 動 物 可 能 通 過 它 們 的 平 衡 器 官 感 覺 到 ； 一 種 振 動 異 常 ， 某 些 動 物 的 聽 覺 器 官 也 許 能 夠 察 覺 出 來 。 地 震 前 地 下 岩 層 早 已 在 逐 日 緩 慢 活 動 ， 而 斷 層 面 之 間 又 具 有 強 大 的 摩 擦 力 。 這 種 摩 擦 力 會 產 生 一 種 低 於 人 的 聽 覺 所 能 感 覺 到 的 低 頻 聲 波 。 人 對 每 秒 20 次 以 上 的 聲 波 才 能 感 覺 到 ， 而 動 物 則 不 然 。 那 些 感 覺 十 分 靈 敏 的 動 物 ， 在 感 觸 到 這 種 低 聲 波 時 ， 便 會 驚 恐 萬 狀 ， 以 至 出 現 冬 蛇 出 洞 、 魚 躍 水 面 等 異 常 現 象 。 [SEP] 動 物 的 器 官 感 覺 與 人 的 相 比 有 什 麼 不 同? [SEP] 沒 有 人 的 靈 敏 [SEP]'

In [12]:
tokenized_data_ds = {2:tokenized_data_ds2, 3:tokenized_data_ds3, 4:tokenized_data_ds4}
tokenized_data_ds[2]

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'label', 'choices', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 553
    })
    dev: Dataset({
        features: ['context', 'question', 'label', 'choices', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 170
    })
})

In [13]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    num_choices: int = 2

    def __call__(self, features):
        labels = [feature.pop("label") for feature in features]
        batch_size = len(features)
        # num_choices = len(features[0]["choices"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(self.num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, self.num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [14]:

accuracy = evaluate.load("accuracy")

In [15]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
# model = BertForMultipleChoice.from_pretrained("hfl/chinese-bert-wwm-ext")

model_dict = {}
for num_choice in range(2,5):
    model_dict[num_choice] = AutoModelForMultipleChoice.from_pretrained("ckiplab/albert-base-chinese")
    # model_dict[num_choice] = AutoModelForMultipleChoice.from_pretrained("hfl/chinese-bert-wwm")




In [17]:
training_args =  [
    TrainingArguments(
    output_dir=f"saved_model_wwm{num_choice}",#
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="adamw_torch",
    learning_rate=1e-5,#
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"
    )
    for num_choice in range(2,5)
]

trainers = [
    Trainer(
        model=model_dict[num_choice],
        args=training_args[num_choice-2],
        train_dataset=tokenized_data_ds[num_choice]['train'],
        eval_dataset=tokenized_data_ds[num_choice]["dev"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer, num_choices = num_choice ),
        compute_metrics=compute_metrics,
    )
    for num_choice in range(2,5) 
]

# for i in range(len(trainers)):
#     trainers[i].train()

# trainers[0].train()
# trainers[1].train()
trainers[2].train()
    

The following columns in the training set don't have a corresponding argument in `AlbertForMultipleChoice.forward` and have been ignored: context, choices, question. If context, choices, question are not expected by `AlbertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5045
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 6310
  Number of trainable parameters = 10548737
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.3861, 'learning_rate': 9.207606973058639e-06, 'epoch': 0.4}
{'loss': 1.2849, 'learning_rate': 8.415213946117275e-06, 'epoch': 0.79}


The following columns in the evaluation set don't have a corresponding argument in `AlbertForMultipleChoice.forward` and have been ignored: context, choices, question. If context, choices, question are not expected by `AlbertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1669
  Batch size = 4
Saving model checkpoint to saved_model_wwm4\checkpoint-1262
Configuration saved in saved_model_wwm4\checkpoint-1262\config.json
Model weights saved in saved_model_wwm4\checkpoint-1262\pytorch_model.bin
tokenizer config file saved in saved_model_wwm4\checkpoint-1262\tokenizer_config.json
Special tokens file saved in saved_model_wwm4\checkpoint-1262\special_tokens_map.json


{'eval_loss': 1.2080864906311035, 'eval_accuracy': 0.4907130017974835, 'eval_runtime': 91.6655, 'eval_samples_per_second': 18.208, 'eval_steps_per_second': 4.56, 'epoch': 1.0}
{'loss': 1.1708, 'learning_rate': 7.622820919175912e-06, 'epoch': 1.19}
{'loss': 1.0991, 'learning_rate': 6.830427892234549e-06, 'epoch': 1.58}
{'loss': 1.067, 'learning_rate': 6.038034865293186e-06, 'epoch': 1.98}


The following columns in the evaluation set don't have a corresponding argument in `AlbertForMultipleChoice.forward` and have been ignored: context, choices, question. If context, choices, question are not expected by `AlbertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1669
  Batch size = 4


In [ ]:
tokenizer.decode(tokenized_data_ds2['train']['input_ids'][0][0])

'[CLS] 昨 天 的 考 試 我 半 個 小 時 就 做 完 了 ， 但 是 很 多 不 應 該 錯 的 都 做 錯 了 ， 因 爲 考 試 的 時 候 覺 得 太 簡 單 了 ， 沒 有 認 真 檢 查 。 [SEP] 昨 天 的 考 試 他 考 得 不 錯 。 [SEP] 正 確 [SEP]'

In [ ]:
test_dataset_dict = {2: test_data_ds2['test'], 3:test_data_ds3['test'], 4:test_data_ds4['test']}

In [ ]:
for feature in test_dataset_dict[2]:
    print(cc.convert(feature['choices'][1]))
    # print(feature.items())
    break


錯誤


In [ ]:
# STOP

In [ ]:
import pandas as pd  
def inference(test_dataset_dict: dict, model_path: list):
    index_lst = []
    pred_lst = []
    for num_choice in range(2,5):
        print(f"START: CHOICE {num_choice}")
        tokenizer = BertTokenizerFast.from_pretrained(model_path[num_choice-2]) #, device_map='auto', load_in_8bit=True)
        model = AlbertForMultipleChoice.from_pretrained(model_path[num_choice-2]).to("cuda") # , device_map='auto', load_in_8bit=True)
        
        for i, feature in enumerate(test_dataset_dict[num_choice]):
            # for k, v in feature.items():
                # feature[k] = feature[k].to("cuda")
                
            prompt = f"{feature['context']}[SEP]{feature['question']}"
            prompt_candidates = [[prompt, feature['choices'][i]] for i in range(num_choice)]
            inputs = tokenizer(prompt_candidates, return_tensors="pt", padding=True, truncation=True)
            outputs = model(**{k: v.unsqueeze(0).to("cuda") for k, v in inputs.items()})
            pred = outputs.logits.argmax().item()
            index_lst.append(test_index[num_choice-2][i])
            pred_lst.append(pred+1) # 
        model = None
    output_dict = {'index': index_lst, 'answer': pred_lst}
    output_df = pd.DataFrame(output_dict) 
    output_df.to_csv("HW3_submission.csv", index=False)

model_path=['saved_model_wwm2\checkpoint-417_loss0.68_acc0.59', 'saved_model_wwm3\checkpoint-208_loss1.05_acc0.46', 'saved_model_wwm4\checkpoint-2524_loss1.14_acc0.57']
inference(test_dataset_dict, model_path)       
# outputs = model_dict[2](tokenized_test_data_ds2['test2']['input_ids']) #, labels=labels)  # batch size is 1

# # the linear classifier still needs to be trained
# loss = outputs.loss
# logits = outputs.logits

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file saved_model_wwm2\checkpoint-417_loss0.68_acc0.59\config.json
Model config AlbertConfig {
  "_name_or_path": "ckiplab/albert-base-chinese",
  "architectures": [
    "AlbertForMultipleChoice"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 101,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 102,
  "gap_size": 0,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embe

START: CHOICE 2


loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file saved_model_wwm3\checkpoint-208_loss1.05_acc0.46\config.json
Model config AlbertConfig {
  "_name_or_path": "ckiplab/albert-base-chinese",
  "architectures": [
    "AlbertForMultipleChoice"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 101,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 102,
  "gap_size": 0,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embe

START: CHOICE 3


loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file saved_model_wwm4\checkpoint-2524_loss1.14_acc0.57\config.json
Model config AlbertConfig {
  "_name_or_path": "ckiplab/albert-base-chinese",
  "architectures": [
    "AlbertForMultipleChoice"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 101,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 102,
  "gap_size": 0,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_emb

START: CHOICE 4


In [ ]:
test_dict = {'index':[1,2,3,4,5], 'answer':[9,8,7,6,5]}
with open("test.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow()
    w.writerow(test_dict)

TypeError: writer.writerow() takes exactly one argument (0 given)